# 1. Necessary Headfiles

In [8]:
import torch
import torchvision
# from torch.utils.tensorboard import SummaryWriter
import time
from torch import nn
from torch.utils.data import DataLoader
from load_data import MyData  # self-made
from torchvision import transforms
from tqdm import tqdm_notebook as tqdm # View procedure
import os
import scipy.io
from random import random
import numpy as np
import gc
from torch.utils.tensorboard import SummaryWriter
from torchnlp.word_to_vector import GloVe
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

# 2. Generate eegmap

## 2.1.1  Produce Datamap

In [5]:
from data_preprocess import generate_eegmap
# origin eeg data
root_dir = "../eegdata"
# distribution index matrix
matrix_index = np.array([
    [0,0,2,2,4,4,6,6,8,8,2,2,4,4,6,6,2,4,6,3,3,5,5,3,3,5,5,3,3,2,2,4,4,6,6,1,1,3,3,5,5,7,7,2,2,4,4,6,6,1,1,3,3,5,5,7,7,0,5],
    [4,6,3,7,3,7,3,7,4,6,1,9,1,9,1,9,5,5,5,4,6,4,6,2,8,2,8,0,10,4,6,4,6,4,6,4,6,3,7,3,7,4,6,2,8,2,8,2,8,3,7,1,9,1,9,3,7,5,5]
])
window_overlap = 0
# set GPU usage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#====================================================================
# # -----------------generate single dir----------------------------
# # experimental dir: conditionA, conditionB, conditionC
# exper_dir = "conditionA"
# # condition_dir: hc, mcs, uws
# condi_dir = "hc"
# dataset = MyData(root_dir, exper_dir, condi_dir) # eg.conditionA/hc
# generate_eegmap(dataset, matrix_index, exper_dir, condi_dir, device)

# --------------------generate all dirs------------------------------
for exper_dir in ['conditionA','conditionB','conditionC']:
    for condi_dir in ['hc','mcs','uws']:
        dataset = MyData(root_dir, exper_dir, condi_dir) # eg.conditionA\hc
        generate_eegmap(dataset, matrix_index, exper_dir, condi_dir, device)

## 2.1.2 Split data into 2400-points eegmap pieces

In [31]:
root_dir = "../data/eegmap_direct"
# exper_dir = "conditionB"
# condi_dir = "hc"
for exper_dir in ['conditionA','conditionB','conditionC']:
    for condi_dir in ['hc','mcs','uws']:
        dataset = MyData(root_dir, exper_dir, condi_dir)
        data_all = []
        for person in tqdm(range(len(dataset))):
            filename = os.path.join(dataset.path, dataset.file_path[person])
            data_map = torch.load(filename)
            print(filename)
            print(data_map.size())

            for i in tqdm(range(data_map.size(0))): # 每个被试切割成2400大小的块保存到data_all中
                data_all.append(data_map[i])

        savepath = f"../data/eegmap_chunks/{exper_dir}/{condi_dir}/{exper_dir}_{condi_dir}.pt"
        torch.save(torch.stack(data_all),savepath)
        print(len(data_all))
        del data_all
        del dataset
        del data_map
        gc.collect()
        torch.cuda.empty_cache()

D:\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/20 [00:00<?, ?it/s]

../data/eegmap_direct\conditionC\hc\hc1.set.mat.pt
torch.Size([62, 2400, 10, 11])


D:\Anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


  0%|          | 0/62 [00:00<?, ?it/s]

...

../data/eegmap_direct\conditionC\hc\hc8.set.mat.pt
torch.Size([61, 2400, 10, 11])


  0%|          | 0/61 [00:00<?, ?it/s]

../data/eegmap_direct\conditionC\hc\hc9.set.mat.pt
torch.Size([63, 2400, 10, 11])


  0%|          | 0/63 [00:00<?, ?it/s]

1231


# 3. Prepare Datasets

In [6]:
exper_dir = "conditionC"
# exper_dir = "rest"
hc_path = f"../data/eegmap_chunks/{exper_dir}/hc/{exper_dir}_hc.pt"
mcs_path = f"../data/eegmap_chunks/{exper_dir}/mcs/{exper_dir}_mcs.pt"
uws_path = f"../data/eegmap_chunks/{exper_dir}/uws/{exper_dir}_uws.pt"
hc = torch.load(hc_path)
mcs = torch.load(mcs_path)
uws = torch.load(uws_path)
print(hc.size())
print(mcs.size())
print(uws.size())

torch.Size([1231, 2400, 10, 11])
torch.Size([1353, 2400, 10, 11])
torch.Size([1220, 2400, 10, 11])


## 3.1 Split Map Data - Cross Fold Valid

In [ ]:
# 定义函数用于保存数据集
def save_dataset(data, exper_dir, subset, mode, fold):
    torch.save(data, f"../data/eegmap_split/{exper_dir}/train/{subset}/{exper_dir}_{subset}_{mode}_{fold}.pt")

# test dataset
# 生成随机种子
torch.manual_seed(32)
# 生成随机索引
hc_indices = torch.randperm(len(hc))
mcs_indices = torch.randperm(len(mcs))
uws_indices = torch.randperm(len(uws))
test_percentage = 0.1
# hc
print(len(hc))
test_end = int(test_percentage * len(hc))
test_hc_indices = hc_indices[:test_end]
test_hc = hc[test_hc_indices]
torch.save(test_hc, f"../data/eegmap_split/{exper_dir}/test/hc/{exper_dir}_hc_test.pt")
hc = hc[hc_indices[test_end:]]
print(len(hc))
# mcs
print(len(mcs))
test_end = int(test_percentage * len(mcs))
test_mcs_indices = mcs_indices[:test_end]
test_mcs = mcs[test_mcs_indices]
torch.save(test_mcs, f"../data/eegmap_split/{exper_dir}/test/mcs/{exper_dir}_mcs_test.pt")
mcs = mcs[mcs_indices[test_end:]]
print(len(mcs))
#uws
print(len(uws))
test_end = int(test_percentage * len(uws))
test_uws_indices = uws_indices[:test_end]
test_uws = uws[test_uws_indices]
torch.save(test_uws, f"../data/eegmap_split/{exper_dir}/test/uws/{exper_dir}_uws_test.pt")
uws = uws[uws_indices[test_end:]]
print(len(uws))
# 清理内存
del hc_indices, mcs_indices, uws_indices, test_hc_indices, test_mcs_indices, test_uws_indices
del test_hc, test_mcs, test_uws
gc.collect()
torch.cuda.empty_cache()

# train dataset
# 生成随机索引
hc_indices = torch.randperm(len(hc))
mcs_indices = torch.randperm(len(mcs))
uws_indices = torch.randperm(len(uws))
fold_num = 5
# 划分数据集并保存
for fold in tqdm(range(fold_num)):
    # 划分 hc 数据集
    len_hc = len(hc)
    val_size = len_hc // fold_num  # 每一折的验证集大小
    val_start = fold * val_size  # 验证集起始索引
    val_end = min((fold + 1) * val_size, len_hc)  # 验证集结束索引
    train_hc_indices = torch.cat([hc_indices[:val_start], hc_indices[val_end:]])
    val_hc_indices = hc_indices[val_start:val_end]
    train_hc = hc[train_hc_indices]
    val_hc = hc[val_hc_indices]
    save_dataset(train_hc, exper_dir, 'hc', 'train', fold)
    save_dataset(val_hc, exper_dir, 'hc', 'val', fold)
    print(train_hc.size())
    print(val_hc.size())
    
    # 划分 mcs 数据集
    len_mcs = len(mcs)
    val_size = len_mcs // fold_num
    val_start = fold * val_size
    val_end = min((fold + 1) * val_size, len_mcs)
    train_mcs_indices = torch.cat([mcs_indices[:val_start], mcs_indices[val_end:]])
    val_mcs_indices = mcs_indices[val_start:val_end]
    train_mcs = mcs[train_mcs_indices]
    val_mcs = mcs[val_mcs_indices]
    save_dataset(train_mcs, exper_dir, 'mcs', 'train', fold)
    save_dataset(val_mcs, exper_dir, 'mcs', 'val', fold)
    print(train_mcs.size())
    print(val_mcs.size())
    
    # 划分 uws 数据集
    len_uws = len(uws)
    val_size = len_uws // fold_num
    val_start = fold * val_size
    val_end = min((fold + 1) * val_size, len_uws)
    train_uws_indices = torch.cat([uws_indices[:val_start], uws_indices[val_end:]])
    val_uws_indices = uws_indices[val_start:val_end]
    train_uws = uws[train_uws_indices]
    val_uws = uws[val_uws_indices]
    save_dataset(train_uws, exper_dir, 'uws', 'train', fold)
    save_dataset(val_uws, exper_dir, 'uws', 'val', fold)
    print(train_uws.size())
    print(val_uws.size())
    
    # 清理内存
    del train_hc_indices, train_mcs_indices, train_uws_indices,
    del val_hc_indices, val_mcs_indices, val_uws_indices
    del train_hc, train_mcs, train_uws, val_hc, val_mcs, val_uws
    gc.collect()
    torch.cuda.empty_cache()
    
# 清理内存
del hc_indices, mcs_indices, uws_indices
del hc, mcs, uws
gc.collect()
torch.cuda.empty_cache()